# Solving Systems of Linear Equations with Python's Numpy

+ 4x + 3y + 2z = 25
+ -2x + 2y + 3z = -10
+ 3x -5y + 2z = -4

In [2]:
import numpy as np

A = np.array([[4, 3, 2], [-2, 2, 3], [3, -5, 2]])
B = np.array([25, -10, -4])
X = np.linalg.inv(A).dot(B)

print(X)

from enum import Enum
class Action(Enum):
    UP = [-1, 0]
    DOWN = [1, 0]
    LEFT = [0, -1]
    RIGHT = [0, 1]

[ 5.  3. -2.]


In [3]:
#Question 1
#Policy Evalution 
#Bellman equation and liners system solver
Reward = np.arange(16, dtype=float)
Reward = Reward.reshape(16,1)
Reward = np.negative(np.ones_like(Reward))
Reward[0] = 0
Reward[15] = 0

Prob = np.array([0.25, 0.25, 0.25, 0.25,0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25])
#Prob = np.array([0, 1/3, 1/3, 1/2,1/3, 1/4, 1/4, 1/3,1/3, 1/4, 1/4, 1/3,1/2, 1/3, 1/3, 0])
Prob = Prob.reshape(4,4)

Trans = np.arange(16*16, dtype=float)
Trans = Trans.reshape(16, 16)
Trans = np.zeros_like(Trans)

for i in range(16):
    
    for j in range(16):
        if i == 0 or i == 15 :#or j == 0 or j==15:
            Trans[i][j] = 0
        else:
            if j == i-1 and (j%4) != 3:
                Trans[i][j] = Prob[int(i/4)][i%4]
            if j == i+1 and (j%4) != 0:
                Trans[i][j] = Prob[int(i/4)][i%4]
            if j == i-4:
                Trans[i][j] = Prob[int(i/4)][i%4]
            if j == i+4:
                Trans[i][j] = Prob[int(i/4)][i%4]
    if i-4 < 0:
        Trans[i][i] = 0.25
    elif i+4 > 15:
        Trans[i][i] = 0.25
    if i%4 == 0 or i%4 == 3:
        Trans[i][i] = 0.25
    if i == 3 or i == 12:
        Trans[i][i] = 0.5
        
Trans = np.identity(16) - Trans
            
A = np.array(Trans)
B = np.array(Reward)

val = np.linalg.inv(A).dot(B)

val = val.reshape(4, 4)
val = np.round(val, 2)
print(val)


[[  0. -14. -20. -22.]
 [-14. -18. -20. -20.]
 [-20. -20. -18. -14.]
 [-22. -20. -14.   0.]]


In [4]:
#Question 1
#Iterative Policy Evalution 
import math 
def iterative_policy(row, col, reward, discount):

    state = np.arange(row*col, dtype = float)
    val = np.arange(row*col, dtype = float)
    state = state.reshape((row,col))
    val = val.reshape((row,col))
    state = np.zeros_like(state)
    val = np.zeros_like(val)

    while True:
        for i in range(row):
            for j in range(col):
                val[i][j]=0
                if i == 0 and j == 0: 
                    continue
                if i == 3 and j == 3:
                    continue 
                if j-1 >= 0:
                    val[i][j] += (reward + discount*state[i][j-1])
                elif j-1 < 0:
                    val[i][j] += (reward + discount*state[i][j])
                if j+1 <= col-1:
                    val[i][j] += (reward + discount*state[i][j+1])
                elif j+1 > col-1:
                    val[i][j] += (reward + discount*state[i][j])
                if i-1 >= 0:
                    val[i][j] += (reward + discount*state[i-1][j])
                elif i-1 <0:
                    val[i][j] += (reward + discount*state[i][j])
                if i+1 <= row-1:
                    val[i][j] += (reward + discount*state[i+1][j])
                elif i+1 > row-1:
                    val[i][j] += (reward + discount*state[i][j])

                val[i][j] = val[i][j]/4
                val[i][j] = round(val[i][j], 2)

        if np.sum(np.abs(val-state)) < 1e-4:
            break
        state = np.copy(val)        
    print(state)

iterative_policy(4, 4, -1, 1)



[[  0.   -13.96 -19.94 -21.94]
 [-13.96 -17.95 -19.94 -19.94]
 [-19.94 -19.94 -17.95 -13.96]
 [-21.94 -19.94 -13.96   0.  ]]


In [5]:
#Question 2
#Random Policy

def iter_policy_evol(discount):
    state = np.arange(25, dtype = float)
    val = np.arange(25, dtype = float)
    state = state.reshape((5,5))
    val = val.reshape((5,5))
    state = np.zeros_like(state)
    val = np.zeros_like(val)

    while True:
        for i in range(5):
            for j in range(5):
                val[i][j] = 0
                ##Action: UP
                reward, prob, dest = move([i, j], Action.UP.value)
                val[i][j] += prob*(reward + discount*(state[dest[0]][dest[1]]))
                ##Action: DOWN
                reward, prob, dest = move([i, j], Action.DOWN.value)
                val[i][j] += prob*(reward + discount*(state[dest[0]][dest[1]]))
                ##Action: LEFT
                reward, prob, dest = move([i, j], Action.LEFT.value)
                val[i][j] += prob*(reward + discount*(state[dest[0]][dest[1]]))
                ##Action: RIGHT
                reward, prob, dest = move([i, j], Action.RIGHT.value)
                val[i][j] += prob*(reward + discount*(state[dest[0]][dest[1]]))

                val[i][j] = round(val[i][j],2)
        
        if np.sum(np.abs(val-state)) < 1e-4:
            break
        state = np.copy(val)
    
    print(state)

In [9]:

def move(currstate, dir):
    if currstate == [0, 1]:
        reward = 10
        prob = 0.25
        deststate = [4, 1]
    elif currstate == [0,3]:
        reward = 5
        prob = 0.25
        deststate = [2, 3]
    elif currstate[0]+dir[0] < 0 or currstate[0]+dir[0] > 4 or currstate[1]+dir[1] < 0 or currstate[1]+dir[1] > 4:
        reward = -1
        prob = 0.25
        deststate = currstate
    else:
        reward = 0
        prob = 0.25
        deststate = np.array(currstate) + np.array(dir)
    return reward, prob, deststate


In [10]:
## Random Policy
iter_policy_evol(0.9)

[[ 3.32  8.8   4.43  5.32  1.48]
 [ 1.53  3.    2.25  1.91  0.54]
 [ 0.06  0.74  0.68  0.36 -0.4 ]
 [-0.96 -0.43 -0.35 -0.58 -1.17]
 [-1.84 -1.33 -1.22 -1.41 -1.96]]


In [11]:

#Value Iteration for optimal state vale funtion
theta = 1e-4
def value_iteration(discount):
    
    state = np.arange(25, dtype = float)
    val = np.arange(25, dtype = float)
    state = state.reshape((5,5))
    val = val.reshape((5,5))
    state = np.zeros_like(state)
    val = np.zeros_like(val)

    while True:
        for i in range(5):
            for j in range(5):
                val[i][j] = 0
                ##Action: UP
                reward, prob, dest = move([i, j], Action.UP.value)
                val[i][j] = max(val[i][j], reward + discount*(state[dest[0]][dest[1]]))
                ##Action: DOWN
                reward, prob, dest = move([i, j], Action.DOWN.value)
                val[i][j] = max(val[i][j], reward + discount*(state[dest[0]][dest[1]]))
                ##Action: LEFT
                reward, prob, dest = move([i, j], Action.LEFT.value)
                val[i][j] = max(val[i][j], reward + discount*(state[dest[0]][dest[1]]))
                ##Action: RIGHT
                reward, prob, dest = move([i, j], Action.RIGHT.value)
                val[i][j] = max(val[i][j], reward + discount*(state[dest[0]][dest[1]]))
                
                val[i][j] = round(val[i][j],2)
        
        if np.sum(np.abs(val-state)) < theta:
            break
        state = np.copy(val)
    
    print(state)

In [12]:
value_iteration(0.9)

[[21.96 24.4  21.96 19.4  17.46]
 [19.76 21.96 19.76 17.78 16.  ]
 [17.78 19.76 17.78 16.   14.4 ]
 [16.   17.78 16.   14.4  12.96]
 [14.4  16.   14.4  12.96 11.66]]
